In [1]:
from corm import model
import numpy as np
import pandas as pd
from pysb.bng import generate_equations
import os
from sympy import sympify
from helper_fxns import sub_parameters

/sw/lib/python2.7/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
parameters = pd.read_csv('/Users/Erin/Dropbox/Schoolwork/Vanderbilt/Lopez Lab/Project Specific/RS-HDMR/CORM_parameter_sets.csv')

In [3]:
simulation_store = pd.HDFStore('/Users/Erin/git/accre_data_analysis/corm/2016_04_06_COX2_unique_simulations.h5')

In [4]:
simulation_store['unique_simulations_s0'][(0.5, 0.5)][0]

0         0.015
11        0.015
19        0.015
24        0.015
25        0.015
26        0.015
28        0.015
30        0.015
31        0.015
34        0.015
36        0.015
38        0.015
40        0.015
41        0.015
42        0.015
44        0.015
46        0.015
47        0.015
48        0.015
49        0.015
53        0.015
58        0.015
59        0.015
60        0.015
63        0.015
64        0.015
65        0.015
66        0.015
68        0.015
70        0.015
          ...  
349393    0.015
349484    0.015
349510    0.015
349528    0.015
349535    0.015
349536    0.015
349542    0.015
349554    0.015
349572    0.015
349578    0.015
349596    0.015
349615    0.015
349649    0.015
349657    0.015
349669    0.015
349676    0.015
349702    0.015
349706    0.015
349714    0.015
349766    0.015
349818    0.015
349834    0.015
349840    0.015
349845    0.015
349899    0.015
349900    0.015
349907    0.015
349922    0.015
349947    0.015
349980    0.015
Name: 0, dtype: float64

In [5]:
parameters.loc[0]

kcat_AA2       0.858683
kcat_AA3       0.209633
KD_AG_cat3    -3.739227
KD_AG_cat2    -1.646591
KD_AG_allo2    0.222622
KD_AG_allo1    3.322358
KD_AA_allo1    2.992801
KD_AA_allo2    0.130806
KD_AA_allo3   -0.626225
kcat_AG3      -0.735673
KD_AA_cat3    -2.940959
KD_AA_cat2    -3.180799
Name: 0, dtype: float64

In [4]:
generate_equations(model)
model.reactions[21]

{'products': (3, 7),
 'rate': __s10*kcat_AA3,
 'reactants': (10,),
 'reverse': (False,),
 'rule': ('kcat_AA_3',)}

In [5]:
len(model.reactions)

29

In [6]:
flux = np.zeros((len(model.reactions), len(simulation_store['unique_simulations_s0'][(0.5, 0.5)].columns)))

In [8]:
[i for i in model.reactions[9]['rate'].args]

[__s6, kcat_AG1]

In [9]:
[1, 2]+[3,4]

[1, 2, 3, 4]

In [10]:
model.reactions[0]['rate'].subs([('__s0', 1), ('__s2', 2), ('kf_AA_cat1', 3)])

6

In [7]:
params = {param: value for param,value in zip(parameters.columns, parameters.loc[0])}
sub_parameters(model, params)
for rxn in range(len(model.reactions)):
    for tpt in range(len(simulation_store['unique_simulations_s0'][(0.5, 0.5)].columns)):
        rxn_args = model.reactions[rxn]['rate'].args
        species = [str(i) for i in rxn_args if '__' in str(i)]
        params = [str(i) for i in rxn_args if str(i) not in species]
        species_sub = [(i, simulation_store['unique_simulations_'+i[2:]][(0.5, 0.5)][tpt].loc[0]) for i in species]
        params_sub = [(i, model.parameters[i].value) for i in params]
        full_subs = species_sub + params_sub
        print 'full subs: ',full_subs
        flux[rxn][tpt] = model.reactions[rxn]['rate'].subs(full_subs)

full subs:  [('__s0', 0.014999999999999999), ('__s2', 0.5), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0038492504717866829), ('__s2', 0.48066648114663968), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0039171837364276466), ('__s2', 0.47100182000024926), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0039863588229210375), ('__s2', 0.46143999348784992), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0040567833064959011), ('__s2', 0.45198216850159995), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0041284633993399428), ('__s2', 0.44262948879842934), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0042014041904983755), ('__s2', 0.43338308071857268), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0042756094357417193), ('__s2', 0.42424404776241237), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0043510818290962957), ('__s2', 0.41521348064396202), ('kf_AA_cat1', 1000.0)]
full subs:  [('__s0', 0.0044278208191554152), ('__s2', 0.40629243339411819), ('kf_AA_cat1', 1000.0)]
full su

In [16]:
flux[5]

array([ 0.        ,  0.00288417,  0.00287616,  0.00286764,  0.00285858,
        0.002849  ,  0.00283887,  0.00282819,  0.00281694,  0.00280513])

In [19]:
flux[19]

array([ 0.        ,  0.00467767,  0.00462379,  0.00456917,  0.00451381,
        0.00445773,  0.00440094,  0.00434345,  0.00428529,  0.00422648])

In [22]:
flux[21]

array([ 0.        ,  0.00127854,  0.00124935,  0.00122036,  0.00119158,
        0.00116301,  0.00113466,  0.00110655,  0.0010787 ,  0.00105109])

In [25]:
model.reactions[16]

{'products': (9,),
 'rate': __s1*__s5*kf_AG_allo2,
 'reactants': (1, 5),
 'reverse': (False,),
 'rule': ('bind_COX2AA_AG_allo',)}

In [19]:
flux[14]

array([  0.        ,  17.76863085,  17.56348909,  17.35552899,
        17.1447921 ,  16.93132409,  16.71517746,  16.49640881,
        16.27508318,  16.05126752])